### ToDo
- [ ] load stl
- [ ] create bounding box + AABB algo
- [ ] calculate center of mass
- [ ] place 1. stl object into middle (by center of mass)
- [ ] randomly place 2. object:
y ^ 
 | 
 |
------> 
 |    x

- move only x, y and rotate on x-y-plane
- remove all objects which are out of plane (AABB)
- get all distancs of center of mass
- take closest object (by center of mass), check for collision by AABB -> if none use it / if yes -> check for triangle collisions -> if yes take next (repeat) -> if none stop

In [1]:
%pip install numpy plotly scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import struct
import numpy as np

def read_ascii_stl(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    vertices = []
    for line in lines:
        parts = line.split()
        if len(parts) > 0 and parts[0] == 'vertex':
            vertices.append(list(map(float, parts[1:])))
    
    faces = np.array(vertices).reshape((-1, 3, 3))
    return faces, np.unique(faces.reshape(-1, 3), axis=0)

def read_binary_stl(filename):
    with open(filename, 'rb') as f:
        header = f.read(80)
        num_faces = struct.unpack('<I', f.read(4))[0]
        faces = []
        for _ in range(num_faces):
            f.read(12)  # skip the normal vector
            for _ in range(3):  # read and store the vertices
                vertex = struct.unpack('<fff', f.read(12))
                faces.append(vertex)
            f.read(2)  # skip attribute byte count

    faces = np.array(faces).reshape((-1, 3, 3))
    return faces, np.unique(faces.reshape(-1, 3), axis=0)

def read_stl_file(filename):
    with open(filename, 'rb') as f:
        if f.read(5).decode('utf-8').lower() == 'solid':
            return read_ascii_stl(filename)
        else:
            return read_binary_stl(filename)

# Usage
faces, vertices = read_stl_file('cube.stl')
print('Faces:', faces.shape)
print('Vertices:', vertices.shape)

print("Shape 1: ", faces[0].shape)
print("Data 1:", faces[0])
print("Shape 2: ", faces[1].shape)
print("Data 2: ", faces[1])




Faces: (12, 3, 3)
Vertices: (8, 3)
Shape 1:  (3, 3)
Data 1: [[0. 4. 6.]
 [2. 0. 6.]
 [2. 4. 6.]]
Shape 2:  (3, 3)
Data 2:  [[2. 0. 6.]
 [0. 4. 6.]
 [0. 0. 6.]]


In [3]:
import numpy as np
from scipy.spatial import ConvexHull

def is_point_in_tri(point, tri):
    # calculate unit normal vector of the triangle
    u = tri[1] - tri[0]
    v = tri[2] - tri[0]
    n = np.cross(u, v)
    n /= np.linalg.norm(n)

    # project point and triangle onto plane defined by normal vector
    pp = point - np.dot(n, point) * n
    tp = tri - np.dot(n, tri.T).T * n

    # calculate 2D ConvexHull of the triangle points
    hull = ConvexHull(tp[:, :2])

    # check if the projected point is in the ConvexHull
    return all(np.dot(n, pp - tp[hull.vertices]) < 1e-5)

def is_edge_in_tri(edge, tri):
    return is_point_in_tri(edge[0], tri) or is_point_in_tri(edge[1], tri)

def are_triangles_colliding(tri1, tri2):
    # Check if any vertex of one triangle is the same as a vertex in the other triangle
    for point1 in tri1:
        for point2 in tri2:
            if np.allclose(point1, point2):
                return True

    # If no common vertices, check if any of the vertices of one triangle is within the other triangle
    for i in range(3):
        edge = [tri1[i], tri1[(i+1)%3]]
        if is_edge_in_tri(edge, tri2):
            return True
    return False


In [4]:
tri1 = np.array([[0., 4., 6.], [2., 0., 6.], [2., 4., 6.]])
tri2 = np.array([[2., 0., 6.], [0., 4., 6.], [0., 0., 6.]])

print(are_triangles_colliding(tri1, tri2))


True


In [5]:
import plotly.graph_objects as go

def plot_triangles(triangle1, triangle2, title='Triangle Plot'):
    x1, y1, z1 = triangle1.T
    x2, y2, z2 = triangle2.T

    fig = go.Figure(data=[go.Scatter3d(
        x=np.concatenate([x1, [x1[0]]]),
        y=np.concatenate([y1, [y1[0]]]),
        z=np.concatenate([z1, [z1[0]]]),
        mode='lines+markers',
        name='Triangle 1',
        marker=dict(
            size=6,
            color='red',
        ),
        line=dict(
            color='red',
            width=2
        )
    ), go.Scatter3d(
        x=np.concatenate([x2, [x2[0]]]),
        y=np.concatenate([y2, [y2[0]]]),
        z=np.concatenate([z2, [z2[0]]]),
        mode='lines+markers',
        name='Triangle 2',
        marker=dict(
            size=6,
            color='blue',
        ),
        line=dict(
            color='blue',
            width=2
        )
    )])

    fig.update_layout(scene=dict(
                      xaxis_title='X',
                      yaxis_title='Y',
                      zaxis_title='Z'),
                      width=700,
                      margin=dict(r=20, b=10, l=10, t=10),
                      title=title)
    fig.show()


In [6]:
# Define our triangles
triangle1 = np.array([[0., 4., 6.], [2., 0., 6.], [2., 4., 6.]])
triangle2 = np.array([[2., 0., 6.], [0., 4., 6.], [0., 0., 6.]])
triangle3 = np.array([[3., 0., 7.], [5., 3., 7.], [4., 2., 7.]])

# Plot the triangles
plot_triangles(triangle1, triangle2, title='Triangle 1 and Triangle 2')
plot_triangles(triangle1, triangle3, title='Triangle 1 and Triangle 3')

# Test the collision function
print("Are Triangle 1 and Triangle 2 colliding? ", are_triangles_colliding(triangle1, triangle2))
print("Are Triangle 1 and Triangle 3 colliding? ", are_triangles_colliding(triangle1, triangle3))


Are Triangle 1 and Triangle 2 colliding?  True
Are Triangle 1 and Triangle 3 colliding?  True


In [ ]:
import numpy as np

def are_coplanar(triangle1, triangle2):
    # Create a matrix where each row is a vector of one triangle
    matrix = np.vstack((triangle1, triangle2[0]))

    # Subtract each row by the first row
    matrix = matrix - matrix[0]

    # Calculate the determinant of the 3x3 matrix
    # If it is nearly zero, the points are coplanar
    return abs(np.linalg.det(matrix)) < 1e-5

def project_2D(triangle, normal):
    # Determine the index of the largest component of the normal
    index_max = np.argmax(np.abs(normal))
    # Drop that coordinate to project the triangle to 2D
    return np.delete(triangle, index_max, axis=1)



if are_coplanar(triangle1, triangle2):
    normal = np.cross(triangle1[1]-triangle1[0], triangle1[2]-triangle1[0])
    normal /= np.linalg.norm(normal)
    triangle1 = project_2D(triangle1, normal)
    triangle2 = project_2D(triangle2, normal)

edges1 = np.array([triangle1[i]-triangle1[(i+1)%3] for i in range(3)])
edges2 = np.array([triangle2[i]-triangle2[(i+1)%3] for i in range(3)])

edges = np.concatenate((edges1, edges2))

axes = []
for i in range(len(edges)):
    for j in range(i+1, len(edges)):
        cross_product = np.cross(edges[i], edges[j])
        if np.linalg.norm(cross_product) > 1e-5:
            axes.append(cross_product / np.linalg.norm(cross_product))

axes.append(np.cross(edges1[0], edges1[1]) / np.linalg.norm(np.cross(edges1[0], edges1[1])))
axes.append(np.cross(edges2[0], edges2[1]) / np.linalg.norm(np.cross(edges2[0], edges2[1])))

intersection_exists = True
for axis in axes:
    proj1 = np.dot(triangle1, axis)
    proj2 = np.dot(triangle2, axis)

    if np.max(proj1) < np.min(proj2) or np.max(proj2) < np.min(proj1):
        intersection_exists = False
        break

if intersection_exists:
    print("The triangles intersect.")
else:
    print("The triangles don't intersect.")


In [ ]:
triangle1 = np.array([[0., 4., 6.], [2., 0., 6.], [2., 4., 6.]])
triangle2 = np.array([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.]])